In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

os.chdir('..')
os.chdir('results')

: 

In [13]:
def combined_df(dataset='Coffee', model_name='RNNA', reg=True, exp_name=None):

    attack_type = 'fgsm_disc_attack' if reg else 'fgsm_attack'
    exp = '_' + exp_name if exp_name else ''
    
    path = '/'.join([dataset, model_name, 'attack', attack_type + exp])
    
    files = os.listdir(path)
    files = [f for f in files if f.endswith('.csv')]

    pattern = r"(?<==).*?(?=.csv)"

    all_res = pd.DataFrame()

    for file in files:
        alpha = float(re.search(pattern, file)[0])
        # print(path)
        curr_res = pd.read_csv(path + '/' + file, index_col=0)

        curr_res['alpha'] = alpha
        curr_res.columns = [column.strip(' ') for column in curr_res.columns] 

        all_res = pd.concat([all_res, curr_res], axis=0)

    return all_res.reset_index(drop=True)


def aggregated_results(
    dataset='Coffee', 
    model_name='RNNA', 
    reg=True, 
    exp_name='_MODE', 
    k=1, 
    metr='F_EFF_CONC',
    metr_max=True
    ):

    all_df = combined_df(dataset=dataset, model_name=model_name, reg=reg, exp_name=exp_name)

    grouped = all_df.groupby(['eps', 'alpha'])
    result = grouped.apply(lambda x: x.nlargest(k, metr, keep='last') if metr_max else x.nsmallest(k, metr, keep='last'))

    return result

In [9]:
aggregated_results(exp_name="MODE")

Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE
Coffee/RNNA/attack/fgsm_disc_attack_MODE


ACC     ROC      PR      F1     EFF        L1  \
eps   alpha                                                            
0.005 0.001   41    0.6071  0.6077  0.5302  0.5926  0.4074   37.6125   
      0.010   1394  0.5357  0.5410  0.4863  0.5517  0.4483   15.6550   
      0.100   192   0.4643  0.4538  0.4445  0.3478  0.6522   23.9625   
      1.000   815   0.6786  0.6744  0.5888  0.6400  0.3600   22.3589   
      10.000  356   0.9643  0.9667  0.9286  0.9630  0.0370   21.3953   
      100.000 1731  0.9643  0.9667  0.9286  0.9630  0.0370   20.8811   
0.030 0.001   53    0.7143  0.7128  0.6221  0.6923  0.3077   11.6293   
      0.010   1432  0.4286  0.4256  0.4336  0.3846  0.6154   19.4721   
      0.100   253   0.1786  0.1667  0.4643  0.0000  1.0000   96.9814   
      1.000   866   0.2500  0.2333  0.4643  0.0000  1.0000   95.2564   
      10.000  407   0.4643  0.4385  0.4478  0.1176  0.8824   90.0985   
      100.000 1783  0.5357  0.5051  0.4670  0.1333  0.8667   86.3592   
0.100 0.001   1021  0.3929  0.3974  0.4231  0.4138  0.5862   28.6000   
      0.010   1480  0.5357  0.5410  0.4863  0.5517  0.4483   28.6000   
      0.100   257   0.5714  0.5744  0.5068  0.5714  0.4286   28.3286   
      1.000   910   0.0000  0.0000  0.4643  0.0000  1.0000  203.9286   
      10.000  458   0.0000  0.0000  0.4643  0.0000  1.0000  262.4500   
      100.000 1835  0.5357  0.5000  0.4643  0.0000  1.0000  261.9786   

                    ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  \
eps   alpha                                                                
0.005 0.001   41         0.6296        0.6429     0.0700          1.8400   
      0.010   1394       0.5357        0.5357     0.0651          1.7096   
      0.100   192        0.4643        0.4643     0.1131          2.9696   
      1.000   815        0.6786        0.6786     0.1391          3.6549   
      10.000  356        0.9643        0.9643     0.1396          3.6683   
      100.000 1731       0.9643        0.9643     0.1380          3.6262   
0.030 0.001   53         0.7407        0.7500     0.0569          1.4940   
      0.010   1432       0.4286        0.4286     0.0992          2.6052   
      0.100   253        0.1786        0.1786     0.2393          6.2849   
      1.000   866        0.2500        0.2500     0.3568          9.3728   
      10.000  407        0.4643        0.4643     0.3674          9.6516   
      100.000 1783       0.5357        0.5357     0.3644          9.5727   
0.100 0.001   1021       0.3929        0.3929     0.1372          3.6051   
      0.010   1480       0.5357        0.5357     0.2037          5.3501   
      0.100   257        0.5714        0.5714     0.2642          6.9399   
      1.000   910        0.0000        0.0000     0.7927         20.8223   
      10.000  458        0.0000        0.0000     1.0913         28.6669   
      100.000 1835       0.5357        0.5357     1.0770         28.2913   

                    PROB_HID  ACC_DISC  F1_DISC  ROC_AUC_DISC    CONC  \
eps   alpha                                                             
0.005 0.001   41      0.5564    0.7500   0.6957        0.5753  0.4247   
      0.010   1394    0.1816    0.5000   0.0000        0.2538  0.7462   
      0.100   192     0.0829    0.5179   0.0690        0.0740  0.9260   
      1.000   815     0.0055    0.5000   0.0000        0.0000  1.0000   
      10.000  356     0.0048    0.5000   0.0000        0.0000  1.0000   
      100.000 1731    0.0050    0.5000   0.0000        0.0000  1.0000   
0.030 0.001   53      0.5168    0.7143   0.6364        0.6135  0.3865   
      0.010   1432    0.2189    0.5179   0.0690        0.3469  0.6531   
      0.100   253     0.4161    0.6786   0.5263        0.4171  0.5829   
      1.000   866     0.0050    0.5000   0.0000        0.0000  1.0000   
      10.000  407     0.0008    0.5000   0.0000        0.0000  1.0000   
      100.000 1783    0.0007    0.5000   0.0000        0.0000  1.0000   
0.100 0.001   1021    0.4386    0.6607 

In [10]:
aggregated_results(model_name='ResidualCNN',exp_name="MODE")

Coffee/ResidualCNN/attack/fgsm_disc_attack_MODE
Coffee/ResidualCNN/attack/fgsm_disc_attack_MODE
Coffee/ResidualCNN/attack/fgsm_disc_attack_MODE
Coffee/ResidualCNN/attack/fgsm_disc_attack_MODE
Coffee/ResidualCNN/attack/fgsm_disc_attack_MODE
Coffee/ResidualCNN/attack/fgsm_disc_attack_MODE


TypeError: Column 'F_EFF_CONC' has dtype object, cannot use method 'nlargest' with this dtype

In [11]:
aggregated_results(model_name='TST',exp_name="MODE")

Coffee/TST/attack/fgsm_disc_attack_MODE
Coffee/TST/attack/fgsm_disc_attack_MODE
Coffee/TST/attack/fgsm_disc_attack_MODE
Coffee/TST/attack/fgsm_disc_attack_MODE
Coffee/TST/attack/fgsm_disc_attack_MODE
Coffee/TST/attack/fgsm_disc_attack_MODE


ACC     ROC      PR      F1     EFF        L1  \
eps   alpha                                                           
0.005 0.001   11   0.4643  0.4590  0.4460  0.4000  0.6000   15.5964   
      0.010   631  0.1786  0.1769  0.4148  0.1481  0.8519   25.4921   
      0.100   356  0.0000  0.0000  0.4643  0.0000  1.0000   58.9192   
      1.000   203  0.4643  0.4333  0.4643  0.0000  1.0000   50.9432   
      10.000  815  0.5357  0.5000  0.4643  0.0000  1.0000   51.2460   
      100.000 509  0.5357  0.5000  0.4643  0.0000  1.0000   50.8703   
0.030 0.001   53   0.4286  0.4256  0.4336  0.3846  0.6154   16.9757   
      0.010   666  0.2143  0.2154  0.4033  0.2143  0.7857   24.0878   
      0.100   379  0.2143  0.2000  0.4643  0.0000  1.0000   84.0449   
      1.000   254  0.7143  0.6923  0.6703  0.5556  0.4444   26.8928   
      10.000  866  0.7500  0.7308  0.7115  0.6316  0.3684   26.1986   
      100.000 560  0.7500  0.7308  0.7115  0.6316  0.3684   25.7464   
0.100 0.001   103  0.1071  0.1103  0.4121  0.1379  0.8621   28.6000   
      0.010   715  0.1786  0.1769  0.4148  0.1481  0.8519   28.6000   
      0.100   411  0.3214  0.3000  0.4643  0.0000  1.0000   44.1000   
      1.000   262  0.5714  0.5436  0.4954  0.2500  0.7500   62.5572   
      10.000  916  0.6786  0.6538  0.6291  0.4706  0.5294  288.3213   
      100.000 566  0.7500  0.7308  0.7115  0.6316  0.3684   40.4429   

                   ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  \
eps   alpha                                                               
0.005 0.001   11        0.4815        0.5000     0.0615          1.6150   
      0.010   631       0.1852        0.2143     0.0884          2.3221   
      0.100   356       0.0000        0.0357     0.1683          4.4216   
      1.000   203       0.4815        0.5000     0.1401          3.6809   
      10.000  815       0.5185        0.5000     0.1393          3.6593   
      100.000 509       0.5185        0.5000     0.1395          3.6633   
0.030 0.001   53        0.4444        0.4643     0.0643          1.6881   
      0.010   666       0.2222        0.2500     0.0860          2.2603   
      0.100   379       0.2222        0.2500     0.2418          6.3520   
      1.000   254       0.7037        0.6786     0.1285          3.3756   
      10.000  866       0.7407        0.7143     0.1322          3.4719   
      100.000 560       0.7407        0.7143     0.1301          3.4171   
0.100 0.001   103       0.1111        0.1429     0.0875          2.2977   
      0.010   715       0.1852        0.2143     0.0940          2.4685   
      0.100   411       0.3333        0.3571     0.1267          3.3292   
      1.000   262       0.5926        0.6071     0.1571          4.1279   
      10.000  916       0.6667        0.6429     0.7301         19.1795   
      100.000 566       0.7407        0.7143     0.1132          2.9740   

                   PROB_HID  ACC_DISC  F1_DISC  ROC_AUC_DISC    CONC  \
eps   alpha                                                            
0.005 0.001   11     0.0001    0.5000   0.0000        0.5867  0.4133   
      0.010   631    0.0002    0.5000   0.0000        0.4069  0.5931   
      0.100   356    0.0000    0.5000   0.0000        0.0000  1.0000   
      1.000   203    0.0000    0.5000   0.0000        0.0000  1.0000   
      10.000  815    0.0000    0.5000   0.0000        0.0000  1.0000   
      100.000 509    0.0000    0.5000   0.0000        0.0000  1.0000   
0.030 0.001   53     0.0002    0.5000   0.0000        0.6097  0.3903   
      0.010   666    0.0001    0.5000   0.0000        0.3610  0.6390   
      0.100   379    0.0000    0.5000   0.0000        0.0000  1.0000   
      1.000   254    0.0000    0.5000   0.0000        0.0000  1.0000   
      10.000  866    0.0000    0.5000   0.0000        0.0000  1.0000   
      100.000 560    0.0000    0.5000   0.0000        0.0000  1.0000   
0.100 0.001   103    0.0367    0.5000   0.0000        0.8291  0.1709   
      0.010   715   

In [14]:
aggregated_results(dataset='PowerCons',model_name='ResidualCNN',exp_name="MODE")

ACC       ROC        PR        F1       EFF  \
eps   alpha                                                           
0.005 0.001   509  0.450000  0.450000  0.477647  0.434286  0.565714   
      0.010   50   0.533333  0.533333  0.517982  0.493976  0.506024   
      0.100   662  0.716667  0.716667  0.673333  0.670968  0.329032   
      1.000   356  0.800000  0.800000  0.776563  0.766234  0.233766   
      10.000  815  0.850000  0.850000  0.834783  0.830189  0.169811   
      100.000 203  0.850000  0.850000  0.834783  0.830189  0.169811   
0.010 0.001   546  0.277778  0.277778  0.441799  0.252874  0.747126   
      0.010   101  0.333333  0.333333  0.454545  0.230769  0.769231   
      0.100   712  0.633333  0.633333  0.600000  0.521739  0.478261   
      1.000   407  0.738889  0.738889  0.728723  0.656934  0.343066   
      10.000  865  0.744444  0.744444  0.734259  0.666667  0.333333   
      100.000 253  0.744444  0.744444  0.734259  0.666667  0.333333   
0.030 0.001   611  0.005556  0.005556  0.500000  0.000000  1.000000   
      0.010   152  0.072222  0.072222  0.500000  0.000000  1.000000   
      0.100   755  0.533333  0.533333  0.519298  0.343750  0.656250   
      1.000   438  0.650000  0.650000  0.640323  0.479339  0.520661   
      10.000  899  0.677778  0.677778  0.672549  0.532258  0.467742   
      100.000 291  0.683333  0.683333  0.673423  0.551181  0.448819   

                           L1  ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  \
eps   alpha                                                           
0.005 0.001   509   29.426172     0.500000      0.550000   0.278054   
      0.010   50    23.029573     0.592593      0.633333   0.346739   
      0.100   662   18.376413     0.796296      0.816667   0.471893   
      1.000   356   18.183414     0.870370      0.866667   0.485590   
      10.000  815   18.168802     0.913580      0.894444   0.485694   
      100.000 203   18.176136     0.913580      0.894444   0.485461   
0.010 0.001   546   39.702530     0.308642      0.377778   0.327467   
      0.010   101   39.636528     0.370370      0.433333   0.518891   
      0.100   712   33.813538     0.703704      0.733333   0.813285   
      1.000   407   34.670425     0.783951      0.772222   0.871321   
      10.000  865   34.105537     0.790123      0.777778   0.855717   
      100.000 253   34.079315     0.790123      0.777778   0.855253   
0.030 0.001   611  108.396591     0.006173      0.105556   0.642427   
      0.010   152   98.955948     0.080247      0.172222   0.904416   
      0.100   755   84.026955     0.592593      0.633333   1.970252   
      1.000   438   63.087631     0.685185      0.683333   1.552310   
      10.000  899   67.194290     0.709877      0.700000   1.653750   
      100.000 291   75.866959     0.722222      0.716667   1.864314   

                   ROUGHNESS_NORM  PROB_HID  ACC_DISC   F1_DISC  ROC_AUC_DISC  \
eps   alpha                                                                     
0.005 0.001   509        1.800742  0.035014  0.472222  0.000000      0.094907   
      0.010   50         2.245567  0.007615  0.472222  0.000000      0.020772   
      0.100   662        3.056093  0.000274  0.472222  0.000000      0.000123   
      1.000   356        3.144797  0.000286  0.472222  0.000000      0.000123   
      10.000  815        3.145473  0.000289  0.472222  0.000000      0.000123   
      100.000 203        3.143961  0.000289  0.472222  0.000000      0.000123   
0.010 0.001   546        2.120752  0.058838  0.477778  0.020833      0.160586   
      0.010   101        3.360461  0.015494  0.472222  0.000000      0.044722   
      0.100   712        5.267031  0.000391  0.472222  0.000000      0.000494   
      1.000   407        5.642883  0.000453  0.472222  0.000000      0.000710   
      10.000  865        5.541827  0.000398  0.472222  0.000000      0.000556   
      100.000 253        5.538825  0.000398  0.472222  0.000000      0.000556   
0.030 0.001   611        4.160514  0.162770  0.5

In [15]:
aggregated_results(dataset='PowerCons',model_name='RNNA',exp_name="MODE")

ACC     ROC      PR      F1     EFF        L1  \
eps   alpha                                                           
0.005 0.001   508  0.9000  0.9000  0.8385  0.9072  0.0928   21.9226   
      0.010   50   0.9222  0.9222  0.8716  0.9263  0.0737   19.2533   
      0.100   662  0.9667  0.9667  0.9418  0.9674  0.0326   18.7904   
      1.000   354  0.9889  0.9889  0.9835  0.9889  0.0111   18.0025   
      10.000  815  0.9944  0.9944  0.9890  0.9945  0.0055   18.5614   
      100.000 203  0.9944  0.9944  0.9890  0.9945  0.0055   18.5066   
0.030 0.001   527  0.8278  0.8278  0.7578  0.8394  0.1606   42.7386   
      0.010   81   0.7889  0.7889  0.7298  0.7865  0.2135   53.1323   
      0.100   713  0.7444  0.7444  0.6848  0.7386  0.2614   63.5380   
      1.000   406  0.8167  0.8167  0.7698  0.8070  0.1930   60.8063   
      10.000  861  0.9278  0.9278  0.9123  0.9249  0.0751   55.9890   
      100.000 254  0.9389  0.9389  0.9283  0.9364  0.0636   59.5910   
0.100 0.001   564  0.8833  0.8833  0.8201  0.8912  0.1088   29.4100   
      0.010   106  0.8833  0.8833  0.8280  0.8877  0.1123   30.9122   
      0.100   734  0.6222  0.6222  0.5751  0.6344  0.3656   84.6389   
      1.000   439  0.6333  0.6333  0.5844  0.6333  0.3667  115.5222   
      10.000  905  0.7778  0.7778  0.7236  0.7674  0.2326  123.9133   
      100.000 301  0.8500  0.8500  0.8111  0.8421  0.1579  135.9255   

                   ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  \
eps   alpha                                                               
0.005 0.001   508       0.9050        0.9056     0.2073          1.3426   
      0.010   50        0.9274        0.9278     0.2395          1.5511   
      0.100   662       0.9667        0.9667     0.2596          1.6814   
      1.000   354       0.9889        0.9889     0.2573          1.6664   
      10.000  815       0.9944        0.9944     0.2602          1.6850   
      100.000 203       1.0000        1.0000     0.2609          1.6893   
0.030 0.001   527       0.8324        0.8333     0.2449          1.5859   
      0.010   81        0.7933        0.7944     0.3015          1.9525   
      0.100   713       0.7486        0.7500     0.5287          3.4238   
      1.000   406       0.8212        0.8222     0.6515          4.2196   
      10.000  861       0.9330        0.9333     0.6121          3.9639   
      100.000 254       0.9441        0.9444     0.6581          4.2620   
0.100 0.001   564       0.8833        0.8833     0.2750          1.7810   
      0.010   106       0.8883        0.8889     0.3533          2.2884   
      0.100   734       0.6257        0.6278     0.7107          4.6029   
      1.000   439       0.6369        0.6389     1.2053          7.8059   
      10.000  905       0.7821        0.7833     1.3807          8.9416   
      100.000 301       0.8547        0.8556     1.4674          9.5033   

                   PROB_HID  ACC_DISC  F1_DISC  ROC_AUC_DISC    CONC  \
eps   alpha                                                            
0.005 0.001   508    0.3435    0.6528   0.4939        0.5641  0.4359   
      0.010   50     0.2727    0.6194   0.4170        0.3412  0.6588   
      0.100   662    0.2575    0.6111   0.3966        0.2960  0.7040   
      1.000   354    0.2463    0.6056   0.3826        0.2845  0.7155   
      10.000  815    0.2510    0.6083   0.3896        0.2888  0.7112   
      100.000 203    0.2574    0.6111   0.3966        0.2952  0.7048   
0.030 0.001   527    0.5524    0.7583   0.6947        0.8051  0.1949   
      0.010   81     0.5744    0.7722   0.7172        0.7577  0.2423   
      0.100   713    0.6184    0.7889   0.7432        0.7558  0.2442   
      1.000   406    0.5542    0.7639   0.7038        0.7226  0.2774   
      10.000  861    0.5884    0.7778   0.7260        0.7434  0.2566   
      100.000 254    0.5908    0.7750   0.7216        0.7548  0.2452   
0.100 0.001   564    0.5679    0.7639   0.7038        0.8478  0.1522   
      0.010   106   

In [16]:
aggregated_results(dataset='PowerCons',model_name='TST',exp_name="MODE")

ACC       ROC        PR        F1       EFF  \
eps   alpha                                                           
0.005 0.001   509  0.833333  0.833333  0.768707  0.840426  0.159574   
      0.010   50   0.861111  0.861111  0.806750  0.863388  0.136612   
      0.100   662  0.950000  0.950000  0.944578  0.947977  0.052023   
      1.000   356  0.988889  0.988889  0.983457  0.988889  0.011111   
      10.000  815  0.994444  0.994444  0.989011  0.994475  0.005525   
      100.000 203  0.994444  0.994444  0.989011  0.994475  0.005525   
0.010 0.001   560  0.538889  0.538889  0.521046  0.525714  0.474286   
      0.010   101  0.627778  0.627778  0.581593  0.612717  0.387283   
      0.100   713  0.833333  0.833333  0.788618  0.825581  0.174419   
      1.000   407  0.977778  0.977778  0.972348  0.977528  0.022472   
      10.000  866  0.994444  0.994444  0.989011  0.994475  0.005525   
      100.000 254  0.994444  0.994444  0.989011  0.994475  0.005525   
0.030 0.001   611  0.222222  0.222222  0.457562  0.135802  0.864198   
      0.010   152  0.405556  0.405556  0.464412  0.327044  0.672956   
      0.100   764  0.722222  0.722222  0.659420  0.725275  0.274725   
      1.000   458  0.933333  0.933333  0.904444  0.933333  0.066667   
      10.000  917  0.972222  0.972222  0.951912  0.972678  0.027322   
      100.000 300  0.972222  0.972222  0.951912  0.972678  0.027322   

                           L1  ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  \
eps   alpha                                                           
0.005 0.001   509   34.265842     0.837989      0.838889   0.293708   
      0.010   50    34.323177     0.865922      0.866667   0.312568   
      0.100   662   35.039566     0.955307      0.955556   0.319937   
      1.000   356   35.203125     0.994413      0.994444   0.328234   
      10.000  815   35.220455     1.000000      1.000000   0.327827   
      100.000 203   35.222069     1.000000      1.000000   0.327924   
0.010 0.001   560   65.992393     0.541899      0.544444   0.450410   
      0.010   101   66.353172     0.631285      0.633333   0.485852   
      0.100   713   68.771278     0.837989      0.838889   0.505729   
      1.000   407   69.525063     0.983240      0.983333   0.528544   
      10.000  866   69.619278     1.000000      1.000000   0.528116   
      100.000 254   69.625946     1.000000      1.000000   0.528624   
0.030 0.001   611  177.316559     0.223464      0.227778   1.060134   
      0.010   152  180.008881     0.407821      0.411111   1.168580   
      0.100   764  188.990189     0.726257      0.727778   1.327642   
      1.000   458  194.627869     0.938547      0.938889   1.474577   
      10.000  917  194.844528     0.977654      0.977778   1.489777   
      100.000 300  177.878525     0.977654      0.977778   1.333630   

                   ROUGHNESS_NORM      PROB_HID  ACC_DISC  F1_DISC  \
eps   alpha                                                          
0.005 0.001   509        1.902123  2.953842e-06     0.475      0.0   
      0.010   50         2.024267  1.016868e-06     0.475      0.0   
      0.100   662        2.071989  7.553860e-08     0.475      0.0   
      1.000   356        2.125724  7.057195e-08     0.475      0.0   
      10.000  815        2.123089  7.146026e-08     0.475      0.0   
      100.000 203        2.123712  7.158410e-08     0.475      0.0   
0.010 0.001   560        2.916966  1.038198e-09     0.475      0.0   
      0.010   101        3.146497  4.079800e-10     0.475      0.0   
      0.100   713        3.275223  4.324067e-12     0.475      0.0   
      1.000   407        3.422981  2.045260e-12     0.475      0.0   
      10.000  866        3.420204  2.086205e-12     0.475      0.0   
      100.000 254        3.423499  2.091376e-12     0.475      0.0   
0.030 0.001   611        6.865681  9.531057e-06     0.475      0.0   
      0.010   152        7.568004  2.536359e-32     0.475      0.0   
      0.100   764        8.598126  1.061106e-34     0.475     

In [17]:
aggregated_results(dataset='GunPoint',model_name='ResidualCNN',exp_name="MODE")

ACC     ROC      PR      F1     EFF       L1  \
eps   alpha                                                          
0.005 0.001   5    0.6733  0.6764  0.6032  0.7322  0.2678   2.9183   
      0.010   313  0.4400  0.4429  0.4671  0.5385  0.4615   3.6189   
      0.100   469  0.4667  0.4703  0.4791  0.5789  0.4211   3.2667   
      1.000   199  0.4133  0.4187  0.4564  0.5810  0.4190   4.7534   
      10.000  815  0.8333  0.8355  0.7475  0.8555  0.1445   3.2450   
      100.000 662  0.8933  0.8947  0.8222  0.9024  0.0976   3.2051   
0.030 0.001   52   0.7200  0.7223  0.6420  0.7586  0.2414   4.5000   
      0.010   359  0.2467  0.2495  0.4127  0.3757  0.6243   5.9468   
      0.100   512  0.2467  0.2491  0.4143  0.3616  0.6384   5.4376   
      1.000   208  0.1667  0.1689  0.4103  0.2857  0.7143   8.7748   
      10.000  818  0.3600  0.3613  0.4402  0.4146  0.5854   3.9724   
      100.000 665  0.6333  0.6316  0.5830  0.5736  0.4264   2.6344   
0.100 0.001   152  0.0000  0.0000  0.4933  0.0000  1.0000  56.3867   
      0.010   458  0.0000  0.0000  0.4933  0.0000  1.0000  56.4800   
      0.100   611  0.0000  0.0000  0.4933  0.0000  1.0000  56.5360   
      1.000   305  0.0000  0.0000  0.4933  0.0000  1.0000  56.5080   
      10.000  917  0.0000  0.0000  0.4933  0.0000  1.0000  57.1480   
      100.000 764  0.0000  0.0000  0.4933  0.0000  1.0000  57.2693   

                   ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  \
eps   alpha                                                               
0.005 0.001   5         0.6871        0.6933     0.0316          2.1137   
      0.010   313       0.4490        0.4600     0.0359          2.4031   
      0.100   469       0.4762        0.4867     0.0274          1.8365   
      1.000   199       0.4218        0.4333     0.0237          1.5911   
      10.000  815       0.8503        0.8533     0.0200          1.3407   
      100.000 662       0.9116        0.9133     0.0198          1.3265   
0.030 0.001   52        0.7347        0.7400     0.0461          3.0888   
      0.010   359       0.2517        0.2667     0.0741          4.9627   
      0.100   512       0.2517        0.2667     0.0857          5.7428   
      1.000   208       0.1701        0.1867     0.1459          9.7740   
      10.000  818       0.3673        0.3800     0.0753          5.0461   
      100.000 665       0.6463        0.6533     0.0535          3.5820   
0.100 0.001   152       0.0000        0.0200     0.8896         59.6020   
      0.010   458       0.0000        0.0200     0.8896         59.5967   
      0.100   611       0.0000        0.0200     0.9146         61.2744   
      1.000   305       0.0000        0.0200     0.9308         62.3583   
      10.000  917       0.0000        0.0200     0.9463         63.3988   
      100.000 764       0.0000        0.0200     0.9483         63.5320   

                   PROB_HID  ACC_DISC  F1_DISC  ROC_AUC_DISC    CONC  \
eps   alpha                                                            
0.005 0.001   5      0.6598    0.8200   0.8200        0.8580  0.1420   
      0.010   313    0.6808    0.8033   0.8000        0.8239  0.1761   
      0.100   469    0.4828    0.7033   0.6642        0.6920  0.3080   
      1.000   199    0.3392    0.5100   0.2899        0.4607  0.5393   
      10.000  815    0.2362    0.4100   0.0000        0.2310  0.7690   
      100.000 662    0.2342    0.4100   0.0000        0.2276  0.7724   
0.030 0.001   52     0.8909    0.9100   0.9174        0.9933  0.0067   
      0.010   359    0.9542    0.8867   0.8938        0.9652  0.0348   
      0.100   512    0.9262    0.8733   0.8797        0.9319  0.0681   
      1.000   208    0.9058    0.8667   0.8726        0.9226  0.0774   
      10.000  818    0.8576    0.8400   0.8431        0.8967  0.1033   
      100.000 665    0.7979    0.8333   0.8355        0.8827  0.1173   
0.100 0.001   152    1.0000    0.9100   0.9174        1.0000  0.0000   
      0.010   458    1.0000    0.9100   

In [19]:
aggregated_results(dataset='GunPoint',model_name='RNNA',exp_name="_MODE")


ACC     ROC      PR      F1     EFF       L1  \
eps   alpha                                                          
0.005 0.001   11   0.6533  0.6529  0.5952  0.6389  0.3611   6.3153   
      0.010   320  0.6467  0.6465  0.5889  0.6395  0.3605   6.3126   
      0.100   498  0.5133  0.5133  0.5002  0.5101  0.4899  11.8570   
      1.000   201  0.7867  0.7861  0.7278  0.7746  0.2254  10.6429   
      10.000  815  0.9067  0.9063  0.8756  0.9028  0.0972  10.6753   
      100.000 645  0.9333  0.9331  0.9079  0.9315  0.0685   8.1407   
0.030 0.001   52   0.8133  0.8121  0.7727  0.7910  0.2090   4.5000   
      0.010   359  0.7133  0.7127  0.6509  0.6950  0.3050   5.7716   
      0.100   516  0.6067  0.6070  0.5579  0.6144  0.3856  11.2328   
      1.000   254  0.3000  0.3023  0.4239  0.4000  0.6000  40.1132   
      10.000  866  0.6467  0.6479  0.5860  0.6748  0.3252  35.5456   
      100.000 713  0.8000  0.8005  0.7293  0.8052  0.1948  35.5676   
0.100 0.001   103  0.4667  0.4675  0.4780  0.4937  0.5063  15.0000   
      0.010   410  0.4000  0.4008  0.4526  0.4304  0.5696  15.8400   
      0.100   565  0.3933  0.3940  0.4506  0.4204  0.5796  26.5747   
      1.000   305  0.1800  0.1824  0.4090  0.3051  0.6949  88.4547   
      10.000  917  0.3733  0.3784  0.4410  0.5437  0.4563  88.5920   
      100.000 763  0.4667  0.4728  0.4801  0.6330  0.3670  85.2013   

                   ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  \
eps   alpha                                                               
0.005 0.001   11        0.6853        0.7000     0.0407          2.7252   
      0.010   320       0.6831        0.7000     0.0535          3.5868   
      0.100   498       0.5385        0.5600     0.1093          7.3240   
      1.000   201       0.8369        0.8467     0.1366          9.1534   
      10.000  815       0.9577        0.9600     0.1399          9.3723   
      100.000 645       0.9789        0.9733     0.1062          7.1178   
0.030 0.001   52        0.8531        0.8600     0.0361          2.4177   
      0.010   359       0.7589        0.7733     0.0619          4.1469   
      0.100   516       0.6500        0.6733     0.1184          7.9291   
      1.000   254       0.3169        0.3533     0.3659         24.5166   
      10.000  866       0.6690        0.6733     0.3968         26.5851   
      100.000 713       0.8369        0.8333     0.4084         27.3588   
0.100 0.001   103       0.4965        0.5267     0.0951          6.3724   
      0.010   410       0.4255        0.4600     0.1499         10.0457   
      0.100   565       0.4126        0.4400     0.2493         16.7004   
      1.000   305       0.1915        0.2400     0.9002         60.3067   
      10.000  917       0.3706        0.3800     1.1055         74.0606   
      100.000 763       0.4643        0.4667     1.1229         75.2288   

                   PROB_HID  ACC_DISC  F1_DISC  ROC_AUC_DISC    CONC  \
eps   alpha                                                            
0.005 0.001   11     0.3047    0.6233   0.4488        0.6392  0.3608   
      0.010   320    0.1875    0.5567   0.2811        0.5198  0.4802   
      0.100   498    0.1763    0.5533   0.2717        0.6564  0.3436   
      1.000   201    0.1524    0.5433   0.2431        0.5579  0.4421   
      10.000  815    0.1579    0.5467   0.2527        0.5562  0.4438   
      100.000 645    0.1499    0.5433   0.2431        0.5447  0.4553   
0.030 0.001   52     0.2983    0.6000   0.3939        0.8172  0.1828   
      0.010   359    0.5579    0.7500   0.6914        0.9185  0.0815   
      0.100   516    0.7285    0.8433   0.8266        0.9502  0.0498   
      1.000   254    0.4063    0.6600   0.5278        0.7530  0.2470   
      10.000  866    0.2905    0.6133   0.4257        0.6027  0.3973   
      100.000 713    0.3051    0.6167   0.4335        0.6605  0.3395   
0.100 0.001   103    0.9992    0.9700   0.9709        0.9912  0.0088   
      0.010   410    0.9999    0.9700   

In [20]:
aggregated_results(dataset='GunPoint',model_name='TST',exp_name="MODE")

ACC     ROC      PR      F1     EFF       L1  \
eps   alpha                                                          
0.005 0.001   49   0.6067  0.6058  0.5593  0.5755  0.4245   9.6111   
      0.010   355  0.7867  0.7856  0.7360  0.7647  0.2353   5.1992   
      0.100   482  0.8867  0.8860  0.8553  0.8794  0.1206   1.8135   
      1.000   202  0.9467  0.9463  0.9327  0.9444  0.0556   1.1227   
      10.000  814  0.9733  0.9732  0.9661  0.9726  0.0274   0.8353   
      100.000 662  0.9800  0.9799  0.9730  0.9796  0.0204   0.2617   
0.030 0.001   54   0.6467  0.6454  0.5924  0.6074  0.3926   8.5152   
      0.010   360  0.7200  0.7185  0.6651  0.6818  0.3182   7.4912   
      0.100   512  0.8400  0.8387  0.8080  0.8209  0.1791   3.6388   
      1.000   206  0.8600  0.8588  0.8331  0.8444  0.1556   3.3588   
      10.000  819  0.9000  0.8990  0.8848  0.8905  0.1095   5.9028   
      100.000 671  0.8533  0.8528  0.8088  0.8451  0.1549   5.1724   
0.100 0.001   104  0.4000  0.3992  0.4546  0.3571  0.6429  16.0280   
      0.010   410  0.4800  0.4792  0.4834  0.4429  0.5571  14.8440   
      0.100   563  0.5133  0.5126  0.4998  0.4823  0.5177  14.2333   
      1.000   258  0.2933  0.2934  0.4327  0.2933  0.7067  24.5213   
      10.000  869  0.5200  0.5192  0.5034  0.4857  0.5143  14.1547   
      100.000 716  0.5200  0.5192  0.5034  0.4857  0.5143  14.1493   

                   ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  \
eps   alpha                                                               
0.005 0.001   49        0.6190        0.6267     0.0517          3.4657   
      0.010   355       0.8027        0.8067     0.0364          2.4384   
      0.100   482       0.9048        0.9067     0.0216          1.4438   
      1.000   202       0.9660        0.9667     0.0192          1.2831   
      10.000  814       0.9932        0.9933     0.0179          1.2002   
      100.000 662       1.0000        1.0000     0.0178          1.1908   
0.030 0.001   54        0.6599        0.6667     0.0488          3.2664   
      0.010   360       0.7347        0.7400     0.0457          3.0643   
      0.100   512       0.8571        0.8600     0.0356          2.3820   
      1.000   206       0.8776        0.8800     0.0349          2.3356   
      10.000  819       0.9184        0.9200     0.0383          2.5667   
      100.000 671       0.8707        0.8733     0.0430          2.8788   
0.100 0.001   104       0.4082        0.4200     0.1016          6.8084   
      0.010   410       0.4898        0.5000     0.1001          6.7090   
      0.100   563       0.5238        0.5333     0.0996          6.6715   
      1.000   258       0.2993        0.3133     0.1324          8.8707   
      10.000  869       0.5306        0.5400     0.1014          6.7921   
      100.000 716       0.5306        0.5400     0.1016          6.8061   

                   PROB_HID  ACC_DISC  F1_DISC  ROC_AUC_DISC    CONC  \
eps   alpha                                                            
0.005 0.001   49     0.3775    0.6667   0.5000        0.6943  0.3057   
      0.010   355    0.2316    0.5900   0.3051        0.5895  0.4105   
      0.100   482    0.0693    0.5000   0.0000        0.5152  0.4848   
      1.000   202    0.0661    0.5000   0.0000        0.4861  0.5139   
      10.000  814    0.0666    0.5000   0.0000        0.4924  0.5076   
      100.000 662    0.0687    0.5000   0.0000        0.5069  0.4931   
0.030 0.001   54     0.7994    0.8933   0.8806        0.9238  0.0762   
      0.010   360    0.8344    0.9133   0.9051        0.9448  0.0552   
      0.100   512    0.7357    0.8767   0.8593        0.9480  0.0520   
      1.000   206    0.7247    0.8700   0.8506        0.9500  0.0500   
      10.000  819    0.9220    0.9600   0.9583        0.9645  0.0355   
      100.000 671    0.6541    0.8267   0.7903        0.9431  0.0569   
0.100 0.001   104    0.9104    0.9533   0.9510        0.9695  0.0305   
      0.010   410    0.8947    0.9467   

In [21]:
aggregated_results(dataset='Wafer',model_name='RNNA',exp_name="MODE")

ACC     ROC      PR      F1     EFF        L1  \
eps   alpha                                                           
0.005 0.001   662  0.9714  0.9311  0.9838  0.9840  0.0160    8.4357   
      0.010   203  0.9828  0.9567  0.9897  0.9904  0.0096    5.7874   
      0.100   815  0.9888  0.9666  0.9921  0.9937  0.0063    5.2767   
      1.000   50   0.9925  0.9747  0.9939  0.9958  0.0042    5.2331   
      10.000  349  0.9945  0.9824  0.9958  0.9969  0.0031    4.8653   
      100.000 500  0.9945  0.9830  0.9959  0.9969  0.0031    4.7486   
0.030 0.001   673  0.9534  0.8840  0.9729  0.9739  0.0261   14.4196   
      0.010   224  0.9156  0.7789  0.9494  0.9527  0.0473   21.6832   
      0.100   851  0.8485  0.6540  0.9229  0.9139  0.0861   32.3683   
      1.000   92   0.8840  0.7003  0.9326  0.9350  0.0650   38.9333   
      10.000  405  0.9056  0.7514  0.9435  0.9471  0.0529   43.4290   
      100.000 560  0.9369  0.8238  0.9593  0.9647  0.0353   44.7066   
0.100 0.001   718  0.9127  0.7891  0.9517  0.9509  0.0491   23.1433   
      0.010   259  0.9422  0.8566  0.9667  0.9676  0.0324   24.2932   
      0.100   875  0.8720  0.6685  0.9259  0.9282  0.0718   39.9217   
      1.000   152  0.2284  0.1723  0.8432  0.3606  0.6394  106.6065   
      10.000  417  0.9554  0.8633  0.9681  0.9751  0.0249   47.7358   
      100.000 569  0.9870  0.9590  0.9902  0.9927  0.0073   43.3654   

                   ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  \
eps   alpha                                                               
0.005 0.001   662       0.9762        0.9763     0.1720          1.2649   
      0.010   203       0.9871        0.9872     0.1726          1.2689   
      0.100   815       0.9928        0.9929     0.1739          1.2785   
      1.000   50        0.9967        0.9968     0.1739          1.2789   
      10.000  349       0.9980        0.9977     0.1707          1.2555   
      100.000 500       0.9984        0.9979     0.1698          1.2485   
0.030 0.001   673       0.9569        0.9570     0.2337          1.7187   
      0.010   224       0.9194        0.9197     0.3047          2.2406   
      0.100   851       0.8523        0.8530     0.4491          3.3022   
      1.000   92        0.8875        0.8879     0.5747          4.2261   
      10.000  405       0.9094        0.9096     0.6541          4.8097   
      100.000 560       0.9402        0.9395     0.6819          5.0143   
0.100 0.001   718       0.9170        0.9174     0.3475          2.5555   
      0.010   259       0.9464        0.9466     0.4067          2.9903   
      0.100   875       0.8760        0.8765     0.5812          4.2735   
      1.000   152       0.2295        0.2331     1.1205          8.2396   
      10.000  417       0.9596        0.9598     0.6856          5.0418   
      100.000 569       0.9909        0.9908     0.6623          4.8700   

                   PROB_HID  ACC_DISC  F1_DISC  ROC_AUC_DISC    CONC  \
eps   alpha                                                            
0.005 0.001   662    0.0842    0.5364   0.1423        0.3755  0.6245   
      0.010   203    0.0391    0.5174   0.0747        0.0435  0.9565   
      0.100   815    0.0290    0.5125   0.0562        0.0309  0.9691   
      1.000   50     0.0277    0.5118   0.0538        0.0286  0.9714   
      10.000  349    0.0281    0.5120   0.0544        0.0289  0.9711   
      100.000 500    0.0284    0.5122   0.0550        0.0294  0.9706   
0.030 0.001   673    0.6503    0.8220   0.7846        0.9585  0.0415   
      0.010   224    0.4380    0.7042   0.5823        0.9218  0.0782   
      0.100   851    0.4344    0.7078   0.5896        0.8691  0.1309   
      1.000   92     0.3975    0.6890   0.5513        0.8675  0.1325   
      10.000  405    0.4280    0.7034   0.5807        0.9063  0.0937   
      100.000 560    0.4375    0.7076   0.5891        0.9113  0.0887   
0.100 0.001   718    0.9963    0.9967   0.9967        0.9999  0.0001   
      0.010   259   

In [22]:
aggregated_results(dataset='Wafer',model_name='ResidualCNN',exp_name="MODE")

ACC     ROC      PR      F1     EFF       L1  \
eps   alpha                                                          
0.005 0.001   618  0.5036  0.6980  0.9342  0.6180  0.3820   3.3802   
      0.010   161  0.3756  0.6124  0.9156  0.4701  0.5299   3.5769   
      0.100   775  0.4093  0.6425  0.9222  0.5104  0.4896   3.1062   
      1.000   20   0.3404  0.6125  0.9159  0.4180  0.5820   4.0901   
      10.000  356  0.3637  0.6328  0.9205  0.4482  0.5518   4.8271   
      100.000 509  0.4609  0.6906  0.9330  0.5683  0.4317   4.4889   
0.030 0.001   665  0.2333  0.5082  0.8937  0.2684  0.7316   6.9222   
      0.010   206  0.2790  0.5556  0.9035  0.3343  0.6657   6.0559   
      0.100   818  0.3173  0.5943  0.9119  0.3866  0.6134   4.3920   
      1.000   53   0.4338  0.6714  0.9288  0.5373  0.4627   2.8495   
      10.000  405  0.2773  0.5877  0.9108  0.3214  0.6786   8.1055   
      100.000 558  0.3149  0.6101  0.9157  0.3783  0.6217   7.1864   
0.100 0.001   764  0.0154  0.0714  0.8921  0.0000  1.0000  79.1647   
      0.010   305  0.0376  0.1744  0.8921  0.0000  1.0000  79.8581   
      0.100   917  0.0685  0.3173  0.8921  0.0000  1.0000  80.4172   
      1.000   152  0.0887  0.4113  0.8921  0.0000  1.0000  80.5626   
      10.000  458  0.0991  0.4594  0.8921  0.0000  1.0000  80.3569   
      100.000 611  0.1045  0.4842  0.8921  0.0000  1.0000  74.9401   

                   ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  \
eps   alpha                                                               
0.005 0.001   618       0.5055        0.5073     0.1721          1.2653   
      0.010   161       0.3770        0.3793     0.1712          1.2585   
      0.100   775       0.4108        0.4130     0.1596          1.1739   
      1.000   20        0.3416        0.3441     0.1602          1.1779   
      10.000  356       0.3649        0.3671     0.1391          1.0227   
      100.000 509       0.4618        0.4630     0.1352          0.9942   
0.030 0.001   665       0.2342        0.2370     0.2292          1.6857   
      0.010   206       0.2801        0.2828     0.2217          1.6301   
      0.100   818       0.3185        0.3211     0.1962          1.4425   
      1.000   53        0.4354        0.4375     0.1654          1.2164   
      10.000  405       0.2773        0.2790     0.1729          1.2715   
      100.000 558       0.3151        0.3167     0.1633          1.2007   
0.100 0.001   764       0.0155        0.0191     1.5711         11.5531   
      0.010   305       0.0378        0.0414     1.5787         11.6087   
      0.100   917       0.0687        0.0722     1.5852         11.6565   
      1.000   152       0.0891        0.0925     1.5811         11.6267   
      10.000  458       0.0995        0.1029     1.5738         11.5726   
      100.000 611       0.1044        0.1072     1.4301         10.5157   

                   PROB_HID  ACC_DISC  F1_DISC  ROC_AUC_DISC    CONC  \
eps   alpha                                                            
0.005 0.001   618    0.0398    0.5002   0.0016        0.8464  0.1536   
      0.010   161    0.0273    0.5002   0.0019        0.7243  0.2757   
      0.100   775    0.0030    0.4998   0.0003        0.5195  0.4805   
      1.000   20     0.0010    0.4998   0.0000        0.3387  0.6613   
      10.000  356    0.0002    0.4998   0.0000        0.0123  0.9877   
      100.000 509    0.0002    0.4998   0.0000        0.0121  0.9879   
0.030 0.001   665    0.5775    0.8017   0.7528        0.9779  0.0221   
      0.010   206    0.4914    0.7729   0.7063        0.9336  0.0664   
      0.100   818    0.2054    0.5986   0.3299        0.8822  0.1178   
      1.000   53     0.0111    0.4998   0.0003        0.8203  0.1797   
      10.000  405    0.0060    0.4998   0.0000        0.6366  0.3634   
      100.000 558    0.0054    0.4998   0.0000        0.6232  0.3768   
0.100 0.001   764    0.9879    0.9930   0.9930        0.9993  0.0007   
      0.010   305    0.9718    0.9839   

In [ ]:
def vizualisation(col):
    df = aggregated_results(dataset='Wafer',model_name='ResidualCNN',exp_name="MODE")
    df1 = df.drop(columns=['eps'])
    print(df1)

    

: 

In [31]:
vizualisation('eps')

<bound method Series.unique of eps    alpha       
0.005  0.001    618    0.005
       0.010    161    0.005
       0.100    775    0.005
       1.000    20     0.005
       10.000   356    0.005
       100.000  509    0.005
0.030  0.001    665    0.030
       0.010    206    0.030
       0.100    818    0.030
       1.000    53     0.030
       10.000   405    0.030
       100.000  558    0.030
0.100  0.001    764    0.100
       0.010    305    0.100
       0.100    917    0.100
       1.000    152    0.100
       10.000   458    0.100
       100.000  611    0.100
Name: eps, dtype: float64>
